# Malicious scenario

In the malicious scenario, we do not use an MPC protocol due to the very high computational cost. We however can rely on Alice computing the model inference offline to obtain y' (prediction on x), then participate in the MPC protocol to compute the loss. Given that Alice is malicious, we ask for a ZKP to verify that the prediction is correct.

First, Bob will send his data point (without label) to Alice. Alice runs inference on her model and sends the hash of the prediction, with a ZKP, back to Bob. 

# Part 0: The Setup

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import ezkl
import os
import json

In [2]:
class LeNet(nn.Sequential):
    """
    Adaptation of LeNet that uses ReLU activations
    """

    # network architecture:
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.act = nn.Sigmoid()

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        x = self.act(x)
        return x


circuit = LeNet()

#Next, we define the data loader for CIFAR-10 dataset.
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=False,transform=transform)
data, lbl  = trainset[4]
classes = trainset.classes  # ['airplane', 'automobile', 'bird', ..., 'truck']
class_name = classes[lbl]

label_eye = torch.eye(10)
y = label_eye[lbl]

x = data.unsqueeze(0)  # Add batch dimension

In [3]:
#Specifying some path parameters
model_path = os.path.join('data','network.onnx')
compiled_model_path = os.path.join('data','network.compiled')
pk_path = os.path.join('data','test.pk')
vk_path = os.path.join('data','test.vk')
settings_path = os.path.join('data','settings.json')

witness_path = os.path.join('data','witness.json')
data_path = os.path.join('data','input.json')
output_path = os.path.join('data','output.json')
label_path = os.path.join('data','label.json')

In [4]:
# Flips the neural net into inference mode
circuit.eval()

out = circuit(x)
print(out)


    # Export the model
torch.onnx.export(circuit,               # model being run
                      x,                   # model input (or a tuple for multiple inputs)
                      model_path,            # where to save the model (can be a file or file-like object)
                      export_params=True,        # store the trained parameter weights inside the model file
                      opset_version=10,          # the ONNX version to export the model to
                      do_constant_folding=True,  # whether to execute constant folding for optimization
                      input_names = ['input'],   # the model's input names
                      output_names = ['output'], # the model's output names
                      dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                    'output' : {0 : 'batch_size'}})

data_array = ((x).detach().numpy()).reshape([-1]).tolist()

data = dict(input_data = [data_array])

    # Serialize data into file:
json.dump( data, open(data_path, 'w' ))

out_array = out.detach().numpy().tolist()
output = dict(output_data = out_array)

    # Serialize data into file:
json.dump( output, open(output_path, 'w' ))

label = y.detach().numpy().tolist()
label = dict(label = label)

json.dump( label, open(label_path, 'w' ))

tensor([[0.4969, 0.4906, 0.5051, 0.4835, 0.4953, 0.4783, 0.4900, 0.4970, 0.5006,
         0.4812]], grad_fn=<SigmoidBackward0>)


# Part 1: The ZKP

In [5]:
py_run_args = ezkl.PyRunArgs()
py_run_args.input_visibility = "public" #Bob can see this
py_run_args.output_visibility = "hashed/public" #This hash is given to Bob
py_run_args.param_visibility = "private" 

res = ezkl.gen_settings(model_path, settings_path, py_run_args=py_run_args)
assert res == True


In [6]:
cal_path = os.path.join('data',"calibration.json")

#Alice should use some real data to calibrate the model, here we use random data
data_array = (trainset.data[:10]).reshape([-1]).tolist()

data = dict(input_data = [data_array])

# Serialize data into file:
json.dump(data, open(cal_path, 'w'))


await ezkl.calibrate_settings(cal_path, model_path, settings_path, "resources")
res = ezkl.compile_circuit(model_path, compiled_model_path, settings_path)
assert res == True

# srs path - This actually requires a trusted setup.
res = await ezkl.get_srs( settings_path)
res = await ezkl.gen_witness(data_path, compiled_model_path, witness_path)
assert os.path.isfile(witness_path)

[tensor] decomposition error: integer -387676842 is too large to be represented by base 16384 and n 2
forward pass failed: "failed to forward: [halo2] General synthesis error"
[tensor] decomposition error: integer -581581482 is too large to be represented by base 16384 and n 2
forward pass failed: "failed to forward: [halo2] General synthesis error"
[tensor] decomposition error: integer 535494400 is too large to be represented by base 16384 and n 2
forward pass failed: "failed to forward: [halo2] General synthesis error"
[tensor] decomposition error: integer -1163664725 is too large to be represented by base 16384 and n 2
forward pass failed: "failed to forward: [halo2] General synthesis error"
[tensor] decomposition error: integer -2328279722 is too large to be represented by base 16384 and n 2
forward pass failed: "failed to forward: [halo2] General synthesis error"
[tensor] decomposition error: integer -569667845 is too large to be represented by base 16384 and n 2
forward pass fail

In [7]:
res = ezkl.setup(
        compiled_model_path,
        vk_path,
        pk_path,
    )

assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)
assert os.path.isfile(settings_path)

In [8]:
# GENERATE A PROOF
proof_path = os.path.join('data','test.pf')

res = ezkl.prove(
        witness_path,
        compiled_model_path,
        pk_path,
        proof_path,
        "single",
    )

print(res)
assert os.path.isfile(proof_path)

{'instances': [['...']], 'proof': '0x0e67a0f8fb879b3663ed049dafbc8dd92d00de34d080d1efe8af4c1b0e7ea3bf0ce46f8c96f9f4a24a1bd196db64c7b1186d017d9e43d64b3fff2e19767809ea026ebb7478aa3a752d9a76f5fe2e58f29b63fdbb8ca9e4e66814d3e56fec46e22de0b620356a6dba2d9fbb2cc0ae5e2d59c6e1362884d122d393ce60a19e90db13fa4965725fd26ee735c3a9deb99a5938aec49ce8bb19dd8e4e3c486e0491a8281d3f6a5165639f63b625789ae89f674cc6a5ce9912f6f94558360a0a8de33124cdd8b851cfcf142cf13f7c56be43e340e0e497ac5e46f1f4d4022e7a895f9a0ebfd80a600cc605e13c5ae40785aebd6f381b8c8be4402a76c71b5a3734c140074bfedffc893dbe4372335f19c077c7fc6eeace79362a1490655a99eb54c17011e77faf806088e75a77378ed80961a3209b711286d86a67a2100b81a22bc25400170b86110c1c8f7a915d464ba138a007dd5b46d82b03933e289aeb97db5dd42a47008925ba1411fb01251e8af8615f22ca4145166fecdea7826a0940fe732a20af7cecea54e116f76cbec5a6ef8d22da179221451a4eacad1a0fda730fd6fa211a7bda3ada0b93992686548e428ffa61dc6bbca44e9b7c44182fe5447e374715d9f1ef1c8f113d416c4bc612f829110ac742c95d8fefe41dcc2a8a77cb75ea1e6

In [ ]:
# VERIFY IT

res = ezkl.verify(
        proof_path,
        settings_path,
        vk_path,
        
    )

assert res == True
print("verified")

verified


# Step 2: Secure MPC to calculate valuation

After Alice has computed $y'$, Alice and Bob can then compute the loss function in a secure MPC protocol.
The MPC protocol supposedly requires verifying the Poseidon hash. But we skipped this for now.

In [10]:
#Here is the plain text computation of squared loss
mpc_pred = out.detach().numpy()[0]
mpc_gt = y.detach().numpy()
sq_loss = 0
for i in range(10):
    sq_loss += (mpc_pred[i] - mpc_gt[i])**2
print(sq_loss)

2.439726


In [11]:
# Here is the plain text computation for cross entropy loss
import numpy as np

mpc_pred = out.detach().numpy()[0]
mpc_gt = y.detach().numpy()

cross_entropy_loss = -1 * np.log(mpc_pred[np.argmax(mpc_gt)])
print(cross_entropy_loss)

0.7134389
